# Lab02: Frequent itemset mining

- Student ID: 21424075  
- Student name: Trần Đình Huy

**How to do your homework**


You will work directly on this notebook; the word `TODO` indicate the parts you need to do.

You can discuss ideas with classmates as well as finding information from the internet, book, etc...; but *this homework must be your*.

**How to submit your homework**

Before submitting, rerun the notebook (`Kernel` ->` Restart & Run All`).

Then create a folder named `ID` (for example, if your ID is 1234567, then name the folder `1234567`) Copy file notebook to this folder, compress and submit it on moodle.

**Contents:**

- Frequent itemset mining.

# 1. Preliminaries
## This is how it all started ...
- Rakesh Agrawal, Tomasz Imielinski, Arun N. Swami: Mining Association Rules between Sets of Items in Large Databases. SIGMOD Conference 1993: 207-216
- Rakesh Agrawal, Ramakrishnan Srikant: Fast Algorithms for Mining Association Rules in Large Databases. VLDB 1994: 487-499

**These two papers are credited with the birth of Data Mining**
## Frequent itemset mining (FIM)

Find combinations of items (itemsets) that occur frequently.
## Applications
- Items = products, transactions = sets of products someone bought in one trip to the store.
$\Rightarrow$ items people frequently buy together.
    + Example: if people usually buy bread and coffee together, we run a sale of bread to attract people attention and raise price of coffee.
- Items = webpages, transactions = words. Unusual words appearing together in a large number of documents, e.g., “Brad” and “Angelina,” may indicate an interesting relationship.
- Transactions = Sentences, Items = Documents containing those sentences. Items that appear together too often could represent plagiarism.

## Transactional Database
A transactional database $D$ consists of $N$ transactions: $D=\left\{T_1,T_2,...,T_N\right\}$. A transaction $T_n \in D (1 \le n \le N)$ contains one or more items and that $I= \left\{ i_1,i_2,…,i_M \right\}$ is the set of distinct items in $D$, $T_n \subset I$. Commonly, a transactional database is represented by a flat file instead of a database system: items are non-negative integers, each row represents a transaction, items in a transaction separated by space.

Example: 

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 

30 31 32 

33 34 35 

36 37 38 39 40 41 42 43 44 45 46 

38 39 47 48 

38 39 48 49 50 51 52 53 54 55 56 57 58 

32 41 59 60 61 62 

3 39 48 

63 64 65 66 67 68 



# Definition

- Itemset: A collection of one or more items.
    + Example: {1 4 5}
- **k-itemset**: An itemset that contains k items.
- Support: Frequency of occurrence of an itemset.
    + Example: From the example above, item 3 appear in 2 transactions so its support is 2.
- Frequent itemset: An itemset whose support is greater than or equal to a `minsup` threshold

# The Apriori Principle
- If an itemset is frequent, then all of its subsets must also be frequent.
- If an itemset is not frequent, then all of its supersets cannot be frequent.
- The support of an itemset never exceeds the support of its subsets.
$$ \forall{X,Y}: (X \subseteq Y) \Rightarrow s(X)\ge s(Y)$$


# 2. Implementation


## The Apriori algorithm
Suppose:

$C_k$ candidate itemsets of size k.

$L_k$ frequent itemsets of size k.

The level-wise approach of Apriori algorithm can be descibed as follow:
1. k=1, $C_k$ = all items.
2. While $C_k$ not empty:
    3. Scan the database to find which itemsets in $C_k$ are frequent and put them into $L_k$.
    4. Use $L_k$ to generate a collection of candidate itemsets $C_{k+1}$ of size k+1.
    5. k=k+1.

### Import library

In [691]:
from collections import defaultdict

### Read data
First we have to read data from database

In [692]:

def readData(path):
    """
    Parameters
    --------------------------
        path: path of database D.
         
    --------------------------
    Returns
        data: a dictionary for representing database D
                 - keys: transaction tids
                 - values: itemsets.
        s: support of distict items in D.
    """
    data={}
    s=defaultdict(lambda: 0) # Initialize a dictionary for storing support of items in I.  
    with open(path,'rt') as f:       
        tid=1;
        for line in f:           
            itemset=set(map(int,line.split())) # a python set is a native way for storing an itemset.
            for item in itemset:  
                s[item]+=1     #Why don't we compute support of items while reading data?
            data[tid]= itemset
            tid+=1
   
    return data, s

### Tree Projection

**I gave you pseudo code of Apriori algorithm above but we implement Tree Projection. Tell me the differences of two algorithms.**


**TODO:**

In [693]:
def joinset(a, b):
    '''
    Parameters
    -------------------
        2 itemsets a and b (of course they are at same branch in search space)

    -------------------
    return
        ret: itemset generated by joining a and b
    '''
    # TODO (hint: this function will be called in generateSearchSpace method.):
    combined_set = a.union(b)
    ret = combined_set
    return ret

class TP:
    def __init__(self, data=None, s=None, minSup=None):
        self.data = data
        self.s = {}

        for key, support in sorted(s.items(), key=lambda item: item[1]):
            self.s[key] = support
        # TODO: why should we do this, answer it at the markdown below?
    
        self.minSup = minSup
        self.L = {}  # Store frequent itemsets mined from database
        self.runAlgorithm()

    def initialize(self):
        """
        Initialize search space at first step
        --------------------------------------
        We represent our search space in a tree structure
        """
        tree = {}

        search_space = {}
        for item, support in self.s.items():
            search_space[item] = {}

            search_space[item]['itemset'] = [item]
            ''' 
            python set does not remain elements order
            so we use a list to extend it easily when create new itemset 
            but why we store itemset in data by a python set???? '''
            # TODO: study about python set and its advantages,
            # answer at the markdown below.
 
            search_space[item]['pruned'] = False
            # TODO:
            # After finish implementing the algorithm tell me why should you use this
            # instead of delete item directly from search_space and tree.

            search_space[item]['support'] = support
            tree[item] = {}
            '''
            Why should i store an additional tree (here it called tree)? 
            Answer: This really help in next steps.

            Remember that there is always a big gap from theory to practicality
            and implementing this algorithm in python is not as simple as you think.
            '''

        return tree, search_space

    def computeItemsetSupport(self, itemset):

        '''Return support of itemset'''
        # TODO (hint: this is why i use python set in data)
        support = 0
        itemset = set(itemset)
        for tid, transaction in self.data.items():                         
            if itemset.issubset(transaction):
                support += 1

        return support 

    def get_sub_tree(self, k, tree, search_space, itter_node):
        if k == 0:
            return search_space[itter_node]['support']
        subtree = search_space[itter_node]
        for node in subtree.keys():
            k-=1
            self.get_sub_tree(k,tree,search_space,node)


    def prune(self, k, tree, search_space):

        '''
        In this method we will find out which itemset in current search space is frequent
        itemset then add it to L[k]. In addition, we prune those are not frequent itemsets.
        '''
        if self.L.get(k) is None: self.L[k] = []
        # TODO
        frequent_itemsets = []
        pruned_itemsets = []
        for itemset in search_space.keys():
            
            support = self.computeItemsetSupport(search_space[itemset]['itemset'])
     
            if support >= self.minSup:
                frequent_itemsets.append(itemset)
            else:
                pruned_itemsets.append(itemset)
                search_space[itemset].update({"pruned": True})             
                

        # print(k, frequent_itemsets)
        self.L[k] = self.L[k] + frequent_itemsets
        # for itemset in pruned_itemsets:
        #   del tree[itemset]
        #   search_space[itemset]['pruned'] = True
          
          
       
     


    def generateSearchSpace(self, k, tree, search_space):
        '''
        Generate search space for exploring k+1 itemset. (Recursive function)
        '''
        items = list(tree.keys())
        ''' print search_space.keys() you will understand  
         why we need an additional tree, '''
        l = len(items)
        self.prune(k, tree, search_space)  
        # sub_search_space = {}
        # sub_tree = {}
       
        if l == 0: return  # Stop condition
        for i in range(l - 1):
            sub_search_space = {}
            sub_tree = {}
           
            a = items[i]
                
            if search_space[a]['pruned']: continue
            # print("a" ,search_space[a])
            for j in range(i + 1, l):
                b = items[j]              
                search_space[a][b] = {}
                tree[a][b] = {}
                # You really need to understand what am i doing here before doing work below.
                # (Hint: draw tree and search space to draft).


                # TODO:print
                # First create newset using join set

                newset = joinset(set(search_space[a]['itemset']),set([b]))
               
                # Second add newset to search_space  
                print(len(newset))
                if(len(newset) == )
                search_space[a][b]['itemset'] = newset
                search_space[a][b]['pruned'] = False
                search_space[a][b]['support'] = 0                       
                sub_search_space[tuple(newset)] = search_space[a][b]
                sub_tree[tuple(newset)] = {}
                
            # Generate search_space for k+1-itemset 
            self.generateSearchSpace(k + 1, sub_tree, sub_search_space)
        # self.generateSearchSpace(k + 1, sub_tree, sub_search_space)

    def runAlgorithm(self):
        tree, search_space = self.initialize()  # generate search space for 1-itemset
        self.generateSearchSpace(1, tree, search_space)

    def miningResults(self):
        return self.L

Ok, let's test on a typical dataset `chess`.

In [694]:
data, s = readData('chess.txt')

In [695]:
#
a=TP(data=data,s=s, minSup=3000)
print(a.miningResults())

2
2
2
2
2
2
2
2
2
2
2
3
2
2
2
2
2
2
2
2
2
2
3
3
3
2
2
2
2
2
2
2
2
2
3
3
3
3
3
3
3
2
2
2
2
2
2
2
2
3
3
3
3
3
3
2
2
2
2
2
2
2
3
3
3
3
3
3
3
3
3
3
2
2
2
2
2
2
3
3
3
3
3
3
3
3
3
3
2
2
2
2
2
3
3
3
3
3
3
3
3
3
3
2
2
2
2
3
3
3
3
3
3
2
2
2
3
3
3
2
2
3
2
{1: [48, 56, 66, 34, 62, 7, 36, 60, 40, 29, 52, 58], 2: [(48, 52), (48, 58), (56, 29), (56, 52), (56, 58), (66, 60), (66, 29), (66, 52), (66, 58), (40, 34), (34, 29), (34, 52), (34, 58), (60, 62), (40, 62), (29, 62), (52, 62), (58, 62), (60, 7), (40, 7), (29, 7), (52, 7), (58, 7), (36, 60), (40, 36), (36, 29), (36, 52), (58, 36), (40, 60), (60, 29), (60, 52), (58, 60), (40, 29), (40, 52), (40, 58), (52, 29), (58, 29), (58, 52)], 3: []}


### Answer questions here:
**Why don't we compute support of items while reading data?**

Trong quá trình đọc dữ liệu, chúng ta tính toán hỗ trợ của các mục riêng biệt trong từ điển `s`. Đối với mỗi mục trong mỗi itemset, chúng ta tăng giá trị hỗ trợ của mục đó trong từ điển `s`.

 Mục đích của việc tính toán hỗ trợ ở đây là thu thập thông tin về tần suất xuất hiện của các mục trong cơ sở dữ liệu.

**why should we do sort**

Việc sắp xếp và lưu trữ từ điển `s` theo `support` giúp đảm bảo rằng các mục sẽ được xử lý theo thứ tự tăng dần của `support`. 

Điều này quan trọng vì trong quá trình thực thi thuật toán, việc xử lý các mục theo thứ tự `support` giúp tăng hiệu suất và tính nhất quán của thuật toán.

 Kết quả là các itemset thường xuyên sẽ được khai thác và lưu trữ theo thứ tự hỗ trợ tăng dần trong thuộc tính self.s.

**study about python set and its advantages ?**

`Python set` có một số ưu điểm:

1. Cấu trúc dữ liệu set trong Python giữ được tính duy nhất (unique) của các phần tử. Điều này giúp loại bỏ các phần tử trùng lặp trong itemset, vì các phần tử trong itemset thường không có tính chất thứ tự quan trọng.

2. Set trong Python cung cấp các phương thức và toán tử để thực hiện các phép toán tập hợp như giao, hợp, phần tử không thuộc, kiểm tra sự thuộc, và nhiều hơn nữa. Điều này giúp cho việc xử lý itemset dễ dàng và hiệu quả.

**After finish implementing the algorithm tell me why should you use this? Instead of delete item directly from search_space and tree.**

Lý do sử dụng cờ 'pruned' trong search_space thay vì xóa mục trực tiếp khỏi search_space và tree là để duy trì cấu trúc và tính toàn vẹn của không gian tìm kiếm và cây trong suốt quá trình thực thi thuật toán.

1. Tính linh hoạt: Cờ `pruned` cho phép chúng ta dễ dàng hoàn tác hoặc sửa đổi quyết định cắt tỉa nếu cần thiết. Thay vì xóa các mục một cách vĩnh viễn, chúng ta có thể đánh dấu chúng là đã bị cắt tỉa và sau này xem xét lại nếu cần thiết.Tính linh hoạt này là quan trọng khi triển khai các thuật toán phức tạp, nơi các điều kiện và tiêu chí khác nhau có thể ảnh hưởng đến quá trình cắt tỉa.

2. Tính nhất quán dữ liệu: Không gian tìm kiếm và cây là các thành phần quan trọng của thuật toán, được sử dụng để tạo ra và khám phá các itemset. Bằng cách sử dụng cờ 'pruned', chúng ta có thể giữ cấu trúc gốc nguyên vẹn và đảm bảo rằng các hoạt động và vòng lặp tiếp theo diễn ra đúng. Nếu chúng ta xóa mục trực tiếp, có thể dẫn đến sự không nhất quán hoặc lỗi khi duyệt hoặc truy cập không gian tìm kiếm và cây.


**Apriori algorithm and Tree Projection, tell me the differences of two algorithms.**

- Giải thuật Apriori: Giải thuật Apriori là một phương pháp lặp lại sử dụng việc tạo và loại bỏ ứng viên để tìm tập hợp phổ biến. Nó tạo ra các tập hợp ứng viên có độ dài k dựa trên tập hợp phổ biến có độ dài k-1 và loại bỏ các ứng viên không đáp ứng ngưỡng hỗ trợ tối thiểu.

- Giải thuật Tree Projection: Giải thuật Tree Projection, còn được gọi là FP-growth (Frequent Pattern growth), là một phương pháp chia để trị xây dựng một cấu trúc dữ liệu gọn gọi là FP-tree. Nó xây dựng FP-tree theo cách đệ quy bằng cách chiếu và gộp các tập hợp phổ biến từ tập dữ liệu. Các tập hợp phổ biến có thể được đọc trực tiếp từ FP-tree, loại bỏ nhu cầu tạo và loại bỏ ứng viên.



# 3. Churn analysis

In this section, you will use frequent itemset mining technique to analyze `churn` dataset (for any purposes). 

*Remember this dataset is not represented as a transactional database, first thing that you have to do is transforming it into a flat file.  

In [696]:
import pandas as pd

df = pd.read_csv('churn.txt')
columns = ["Account Length","Int'l Plan", 'VMail Plan', 'CustServ Calls', 'Phone']
data = df[columns].values.tolist()

print(data)

[[128, 'no', 'yes', 1, '382-4657'], [107, 'no', 'yes', 1, '371-7191'], [137, 'no', 'no', 0, '358-1921'], [84, 'yes', 'no', 2, '375-9999'], [75, 'yes', 'no', 3, '330-6626'], [118, 'yes', 'no', 0, '391-8027'], [121, 'no', 'yes', 3, '355-9993'], [147, 'yes', 'no', 0, '329-9001'], [117, 'no', 'no', 1, '335-4719'], [141, 'yes', 'yes', 0, '330-8173'], [65, 'no', 'no', 4, '329-6603'], [74, 'no', 'no', 0, '344-9403'], [168, 'no', 'no', 1, '363-1107'], [95, 'no', 'no', 3, '394-8006'], [62, 'no', 'no', 4, '366-9238'], [161, 'no', 'no', 4, '351-7269'], [85, 'no', 'yes', 1, '350-8884'], [93, 'no', 'no', 3, '386-2923'], [76, 'no', 'yes', 1, '356-2992'], [73, 'no', 'no', 1, '373-2782'], [147, 'no', 'no', 0, '396-5800'], [77, 'no', 'no', 5, '393-7984'], [130, 'no', 'no', 0, '358-1958'], [111, 'no', 'no', 2, '350-2565'], [132, 'no', 'no', 0, '343-4696'], [174, 'no', 'no', 3, '331-3698'], [57, 'no', 'yes', 0, '357-3817'], [54, 'no', 'no', 3, '418-6412'], [20, 'no', 'no', 0, '353-2630'], [49, 'no', 'no'